In [5]:
#!/usr/bin/env python
#__author__ = 'neevsamuel'
import tensorflow as tf
import numpy as np
import time as tm

#Tensor flow executado em compatibilidade com a v1, devido aos comandos utilizados.
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


"""
Data generation for train and test phases
"""
#

#H - Canal
#W - matriz de ruído 3d
#w - vetor de ruído aleatório gaussiano
#H_org - variável de controle pra armazenamento de valor anterior
#K - iterações no tempo

def generate_data(B,K,N,snr_low,snr_high,H_org):
    #H_=np.random.randn(B,N,K)
    W_=np.zeros([B,K,K])    #matriz com zeros
    x_=np.sign(np.random.rand(B,K)-0.5)  
    y_=np.zeros([B,N])
    w=np.random.randn(B,N)  #
    Hy_=x_*0
    H_ = np.zeros([B,N,K])
    HH_= np.zeros([B,K,K])
    SNR_= np.zeros([B])
    for i in range(B):
        #print i
        SNR = np.random.uniform(low=snr_low,high=snr_high)
        H=H_org
        tmp_snr=(H.T.dot(H)).trace()/K  #
        H_[i,:,:]=H
        y_[i,:]=(H.dot(x_[i,:])+w[i,:]*np.sqrt(tmp_snr)/np.sqrt(SNR))
        Hy_[i,:]=H.T.dot(y_[i,:])   #coleta de potência do canal
        HH_[i,:,:]=H.T.dot( H_[i,:,:])
        SNR_[i] = SNR
    return y_,H_,Hy_,HH_,x_,SNR_

#a função retorna com y (canal + ruído), H (canal atualizado), Hy e HH(Densidade de potência) e SNR.
#Dados utilizados para o treinamento do algoritmo.


def affine_layer(x,input_size,output_size,Layer_num):
    W = tf.Variable(tf.random_normal([input_size, output_size], stddev=0.01))
    w = tf.Variable(tf.random_normal([1, output_size], stddev=0.01))
    y = tf.matmul(x, W)+w
    return y

#É retornado y, na forma de ruído.

def relu_layer(x,input_size,output_size,Layer_num):
    y = tf.nn.relu(affine_layer(x,input_size,output_size,Layer_num))
    return y

#o ruído y, é normalizado.Obs: esta função, poderia ser inserida na affline_layer. É utilizada a mesma matriz y.

"""
simulation - parameters:
B - batch size
train_iter - num of train iterations
test_iter -  num of test iteration
low_snr_db_train - the low range of snr in db during train phase
high_snr_db_train - the high range of snr in db during train phase
low_snr_db_test - the low range of snr in db during test phase
high_snr_db_test - the high range of snr in db during test phase
num_snr -  number of SNRs to be tested, equally spaced between low_snr_db_test and high_snr_db_test
fc_size  -  the size of each hidden layer
startingLearningRate - the initial step size of the gradient descent algorithm
decay_factor & decay_step_size - each decay_step_size steps the learning rate decay by decay_factor
H - the channel to detect over
"""
K = 20
N = 30
B = 1000
train_iter = 1000000
test_iter = 1000
low_snr_db_train = 7.0
high_snr_db_train = 14.0
low_snr_db_test = 8.0
high_snr_db_test = 13.0
num_snr = 6
fc_size = 200
num_of_hidden_layers = 4
startingLearningRate = 0.0003
decay_factor = 0.97
decay_step_size = 1000
H=np.genfromtxt('Top06_30_20.csv', dtype=None, delimiter=',')
#end of parameters

low_snr_train = 10.0 ** (low_snr_db_train/10.0)
high_snr_train = 10.0 ** (high_snr_db_train/10.0)
low_snr_test = 10.0 ** (low_snr_db_test/10.0)
high_snr_test = 10.0 ** (high_snr_db_test/10.0)
bers = np.zeros((1,num_snr))

#São coletados os valores de SNR de teste e treinamento em dB

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

#Retorna a variável initial com distribuição truncada de dp 0.05

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#Retorna a variável initial com constante de 0.1 (todos valores), de dimensão shape.
#Vai permitir que os valores obtidos em treinamento sejam editados.

sess = tf.compat.v1.InteractiveSession()

#Rotina de repetição do algoritmo de detecção de MIMO


NNinput = tf.placeholder(tf.float32, shape=[None, N], name='input')
org_siganl = tf.placeholder(tf.float32, shape=[None, K], name='org_siganl')
batchSize = tf.placeholder(tf.int32)

#Coletados valores para as variáveis, pra que seja plotado o gráfico posteriormente.

#The network
h_fc = []
W_fc_input = weight_variable([N, fc_size])
b_fc_input = bias_variable([fc_size])
#os valores são normalizados, e em todas as camadas do DL, a posição de bias assume valor de 0.1

h_fc.append(tf.nn.relu(tf.matmul(NNinput, W_fc_input) + b_fc_input))
#agrega valores de canais otimizados a var h_fc, considerando o melhor desempenho com ruído.


for i in range(num_of_hidden_layers):
    h_fc.append(relu_layer(h_fc[i-1],fc_size, fc_size,'relu'+str(i)))
    
    #realiza o cálculo ("convolução"), para todas as camadas do modelo DL.


W_fc_final = weight_variable([fc_size, K])
b_final= bias_variable([K])
h_final= tf.matmul(h_fc[i], W_fc_final) + b_final

#retorna os valores finais de ruído, bias e canal estimado.

ssd = tf.reduce_sum(tf.square(org_siganl - h_final))
#precisão

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(startingLearningRate, global_step, decay_step_size, decay_factor, staircase=True)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(ssd)

#Ajuste de passo de treinamento e precisão (considerando ssd como melhor resultado)

val = tf.reshape(org_siganl, tf.stack([K * batchSize]))
final = tf.reshape(h_final, tf.stack([K * batchSize]))
rounded = tf.sign(final)
eq = tf.equal(rounded, val)
eq2 = tf.reduce_sum(tf.cast(eq, tf.int32))

#comparação do canal final e inicial. Formatação do canal final em int.


accuracy = ssd

sess.run(tf.global_variables_initializer())
"""
training phase og the network
"""
for i in range(train_iter):
    batch_Y, batch_H, batch_HY, batch_HH, batch_X , SNR1= generate_data(B , K , N , low_snr_train , high_snr_train , H)
    if i % 100 == 0: 
        correct_bits = eq2.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B})
        train_accuracy = accuracy.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B})
        print("step %d, loss is %g, number of correct bits %d" % (i, train_accuracy,correct_bits))
    train_step.run(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B})
    
    #calcular número de bits corretos até que o erro seja 0, ou algo próximo disso.

"""
start testing our net
"""
tmp_bers = np.zeros((1,test_iter))
tmp_times = np.zeros((1,test_iter))
times = np.zeros((1,1))
testHitCount = 0

snr_list_db = np.linspace(low_snr_db_test,high_snr_db_test,num_snr)
snr_list = 10.0**(snr_list_db/10.0)
for i_snr in range (num_snr):
    Cur_SNR = snr_list[i_snr]
    print ('cur snr')
    print (Cur_SNR)
    #medição constante do valor de SNR
    
    for i in range(test_iter):

        batch_Y, batch_H, batch_HY, batch_HH, batch_X , SNR1= generate_data(B , K , N , Cur_SNR , Cur_SNR , H)
        tic = tm.time()
        tmp_bers[0][i] =   eq2.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B}	)
        toc = tm.time()
        tmp_times[0][i] = toc - tic
        if i % 100 == 0:  
            eq2.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B})
            train_accuracy = accuracy.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B})
            print("test accuracy %g" % eq2.eval(feed_dict={NNinput: batch_Y, org_siganl: batch_X, batchSize: B}))
    bers[0][i_snr] = np.mean(tmp_bers[0])
    
    #medição constante do valor de ber
    
times[0][0] = np.mean(tmp_times[0])/B
print ('Average time to detect a single K bit signal is:')
print (times)
bers =  bers/(K*B)
bers = 1- bers
snrdb_list = np.linspace(low_snr_db_test,high_snr_db_test,num_snr)
print('snrdb_list')
print(snrdb_list)
print ('Bit error rates are is:')
print (bers)

#loop que mostra os valores obtidos para o usuário.

step 0, loss is 20218.2, number of correct bits 9952
step 100, loss is 11638.4, number of correct bits 15784
step 200, loss is 4822.45, number of correct bits 19213
step 300, loss is 2769.78, number of correct bits 19785
step 400, loss is 2679.04, number of correct bits 19813
step 500, loss is 2621.79, number of correct bits 19836
step 600, loss is 2564.53, number of correct bits 19837
step 700, loss is 2536.71, number of correct bits 19821
step 800, loss is 2497.58, number of correct bits 19823
step 900, loss is 2444.1, number of correct bits 19805
step 1000, loss is 2223.37, number of correct bits 19825
step 1100, loss is 2086.04, number of correct bits 19837
step 1200, loss is 2072.82, number of correct bits 19816
step 1300, loss is 1898.11, number of correct bits 19814
step 1400, loss is 1728.39, number of correct bits 19854
step 1500, loss is 1655.74, number of correct bits 19834
step 1600, loss is 1599.09, number of correct bits 19819
step 1700, loss is 1424.05, number of correct

step 14400, loss is 412.552, number of correct bits 19887
step 14500, loss is 514.345, number of correct bits 19833
step 14600, loss is 465.027, number of correct bits 19860
step 14700, loss is 496.989, number of correct bits 19854
step 14800, loss is 514.938, number of correct bits 19845
step 14900, loss is 489.498, number of correct bits 19847
step 15000, loss is 585.356, number of correct bits 19824
step 15100, loss is 534.44, number of correct bits 19833
step 15200, loss is 511.495, number of correct bits 19839
step 15300, loss is 456.659, number of correct bits 19854
step 15400, loss is 453.16, number of correct bits 19860
step 15500, loss is 562.533, number of correct bits 19811
step 15600, loss is 457.697, number of correct bits 19853
step 15700, loss is 487.885, number of correct bits 19856
step 15800, loss is 469.79, number of correct bits 19860
step 15900, loss is 471.433, number of correct bits 19857
step 16000, loss is 498.238, number of correct bits 19856
step 16100, loss 

step 28600, loss is 538.392, number of correct bits 19839
step 28700, loss is 393.619, number of correct bits 19891
step 28800, loss is 427.61, number of correct bits 19878
step 28900, loss is 466.521, number of correct bits 19862
step 29000, loss is 534.713, number of correct bits 19838
step 29100, loss is 450.225, number of correct bits 19866
step 29200, loss is 503.715, number of correct bits 19843
step 29300, loss is 431.8, number of correct bits 19860
step 29400, loss is 430.662, number of correct bits 19872
step 29500, loss is 450.297, number of correct bits 19863
step 29600, loss is 491.934, number of correct bits 19854
step 29700, loss is 418.31, number of correct bits 19879
step 29800, loss is 449.861, number of correct bits 19866
step 29900, loss is 405.98, number of correct bits 19879
step 30000, loss is 517.381, number of correct bits 19836
step 30100, loss is 460.881, number of correct bits 19859
step 30200, loss is 443.634, number of correct bits 19858
step 30300, loss is

step 42800, loss is 450.828, number of correct bits 19853
step 42900, loss is 453.889, number of correct bits 19866
step 43000, loss is 416.715, number of correct bits 19875
step 43100, loss is 388.946, number of correct bits 19890
step 43200, loss is 397.519, number of correct bits 19875
step 43300, loss is 453.949, number of correct bits 19866
step 43400, loss is 461.896, number of correct bits 19863
step 43500, loss is 447.636, number of correct bits 19851
step 43600, loss is 381.102, number of correct bits 19888
step 43700, loss is 366.139, number of correct bits 19897
step 43800, loss is 417.022, number of correct bits 19873
step 43900, loss is 385.992, number of correct bits 19889
step 44000, loss is 396.253, number of correct bits 19878
step 44100, loss is 397.771, number of correct bits 19884
step 44200, loss is 470.879, number of correct bits 19862
step 44300, loss is 419.719, number of correct bits 19881
step 44400, loss is 396.064, number of correct bits 19893
step 44500, lo

step 57000, loss is 374.806, number of correct bits 19897
step 57100, loss is 375.463, number of correct bits 19890
step 57200, loss is 369.473, number of correct bits 19892
step 57300, loss is 399.101, number of correct bits 19879
step 57400, loss is 439.982, number of correct bits 19873
step 57500, loss is 413.417, number of correct bits 19880
step 57600, loss is 419.282, number of correct bits 19872
step 57700, loss is 371.489, number of correct bits 19887
step 57800, loss is 431.317, number of correct bits 19866
step 57900, loss is 395.336, number of correct bits 19883
step 58000, loss is 373.861, number of correct bits 19897
step 58100, loss is 421.363, number of correct bits 19881
step 58200, loss is 399.505, number of correct bits 19894
step 58300, loss is 382.948, number of correct bits 19890
step 58400, loss is 361.153, number of correct bits 19897
step 58500, loss is 381.523, number of correct bits 19896
step 58600, loss is 379.214, number of correct bits 19888
step 58700, lo

...

step 939600, loss is 238.075, number of correct bits 19925
step 939700, loss is 231.171, number of correct bits 19927
step 939800, loss is 241.214, number of correct bits 19926
step 939900, loss is 218.297, number of correct bits 19930
step 940000, loss is 262.948, number of correct bits 19922
step 940100, loss is 278.332, number of correct bits 19916
step 940200, loss is 247.023, number of correct bits 19924
step 940300, loss is 252.617, number of correct bits 19920
step 940400, loss is 298.824, number of correct bits 19896
step 940500, loss is 282.953, number of correct bits 19906
step 940600, loss is 222.657, number of correct bits 19935
step 940700, loss is 289.815, number of correct bits 19910
step 940800, loss is 285.171, number of correct bits 19911
step 940900, loss is 345.7, number of correct bits 19893
step 941000, loss is 250.343, number of correct bits 19920
step 941100, loss is 296.526, number of correct bits 19905
step 941200, loss is 258.655, number of correct bits 19927

step 953600, loss is 268.123, number of correct bits 19922
step 953700, loss is 222.986, number of correct bits 19928
step 953800, loss is 257.237, number of correct bits 19921
step 953900, loss is 303.379, number of correct bits 19909
step 954000, loss is 248.264, number of correct bits 19923
step 954100, loss is 266.465, number of correct bits 19917
step 954200, loss is 241.937, number of correct bits 19919
step 954300, loss is 266.648, number of correct bits 19916
step 954400, loss is 209.333, number of correct bits 19930
step 954500, loss is 237.551, number of correct bits 19923
step 954600, loss is 309.833, number of correct bits 19897
step 954700, loss is 268.387, number of correct bits 19920
step 954800, loss is 264.478, number of correct bits 19913
step 954900, loss is 225.961, number of correct bits 19929
step 955000, loss is 188.077, number of correct bits 19944
step 955100, loss is 254.966, number of correct bits 19925
step 955200, loss is 221.344, number of correct bits 199

step 967600, loss is 266.347, number of correct bits 19915
step 967700, loss is 250.496, number of correct bits 19926
step 967800, loss is 295.644, number of correct bits 19910
step 967900, loss is 252.183, number of correct bits 19914
step 968000, loss is 193.173, number of correct bits 19936
step 968100, loss is 259.012, number of correct bits 19910
step 968200, loss is 239.127, number of correct bits 19921
step 968300, loss is 217.054, number of correct bits 19934
step 968400, loss is 224.486, number of correct bits 19928
step 968500, loss is 261.814, number of correct bits 19918
step 968600, loss is 305.623, number of correct bits 19906
step 968700, loss is 182.977, number of correct bits 19946
step 968800, loss is 290.426, number of correct bits 19905
step 968900, loss is 276.501, number of correct bits 19914
step 969000, loss is 316.171, number of correct bits 19892
step 969100, loss is 322.977, number of correct bits 19905
step 969200, loss is 284.647, number of correct bits 199

step 981600, loss is 280.125, number of correct bits 19908
step 981700, loss is 246.415, number of correct bits 19923
step 981800, loss is 318.078, number of correct bits 19907
step 981900, loss is 274.699, number of correct bits 19917
step 982000, loss is 232.272, number of correct bits 19932
step 982100, loss is 307.281, number of correct bits 19907
step 982200, loss is 272.754, number of correct bits 19913
step 982300, loss is 238.641, number of correct bits 19924
step 982400, loss is 222.196, number of correct bits 19929
step 982500, loss is 291.514, number of correct bits 19910
step 982600, loss is 234.791, number of correct bits 19928
step 982700, loss is 294.352, number of correct bits 19902
step 982800, loss is 270.788, number of correct bits 19911
step 982900, loss is 205.732, number of correct bits 19936
step 983000, loss is 283.019, number of correct bits 19916
step 983100, loss is 285.541, number of correct bits 19910
step 983200, loss is 248.775, number of correct bits 199

step 995600, loss is 312.589, number of correct bits 19910
step 995700, loss is 269.471, number of correct bits 19913
step 995800, loss is 229.353, number of correct bits 19931
step 995900, loss is 240.14, number of correct bits 19927
step 996000, loss is 216.418, number of correct bits 19939
step 996100, loss is 219.547, number of correct bits 19938
step 996200, loss is 336.789, number of correct bits 19897
step 996300, loss is 270.709, number of correct bits 19912
step 996400, loss is 229.739, number of correct bits 19928
step 996500, loss is 217.726, number of correct bits 19934
step 996600, loss is 251.695, number of correct bits 19921
step 996700, loss is 279.426, number of correct bits 19912
step 996800, loss is 279.782, number of correct bits 19903
step 996900, loss is 323.48, number of correct bits 19898
step 997000, loss is 243.004, number of correct bits 19928
step 997100, loss is 297.987, number of correct bits 19899
step 997200, loss is 257.461, number of correct bits 19923